http://pandas.pydata.org/pandas-docs/stable/reshaping.html

a couple of examples:

In [7]:
import numpy as np
import pandas as pd
import pandas.util.testing as tm; tm.N = 3
def unpivot(frame):
    N, K = frame.shape
    data = {'value' : frame.values.ravel('F'),
            'variable' : np.asarray(frame.columns).repeat(N),
            'date' : np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])
df = unpivot(tm.makeTimeDataFrame())
df

,date,variable,value
0,2000-01-03,A,0.289665
1,2000-01-04,A,0.036123
2,2000-01-05,A,-0.934871
3,2000-01-03,B,-2.061647
4,2000-01-04,B,1.347385
5,2000-01-05,B,-1.045156
6,2000-01-03,C,-0.480731
7,2000-01-04,C,2.130798
8,2000-01-05,C,-0.624854
9,2000-01-03,D,-0.989817


In [8]:
# reshaping with the pivot function
df2 = df.pivot(index='date', columns='variable', values='value')
df2

variable,A,B,C,D
date,,,,
2000-01-03,0.289665,-2.061647,-0.480731,-0.989817
2000-01-04,0.036123,1.347385,2.130798,-0.364689
2000-01-05,-0.934871,-1.045156,-0.624854,0.334936


### pivot tables
by default, pivot tables aggregate by mean

In [13]:
import datetime

df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 6,
                   'B': ['A', 'B', 'C'] * 8,
                   'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 4,
                   'D': np.random.randn(24),
                   'E': np.random.randn(24),
                   'F': [datetime.datetime(2013, i, 1) for i in range(1, 13)] +
                  [datetime.datetime(2013, i, 15) for i in range(1, 13)]})
df

,A,B,C,D,E,F
0,one,A,foo,-1.214439,-0.540967,2013-01-01
1,one,B,foo,0.875872,-2.614352,2013-02-01
2,two,C,foo,0.015074,1.923733,2013-03-01
3,three,A,bar,1.511884,0.668291,2013-04-01
4,one,B,bar,0.942120,-1.329583,2013-05-01
5,one,C,bar,0.124105,-1.358166,2013-06-01
6,two,A,foo,-1.060059,0.131792,2013-07-01
7,three,B,foo,1.562741,-0.725399,2013-08-01
8,one,C,foo,0.067385,0.581909,2013-09-01
9,one,A,bar,0.209567,1.190426,2013-10-01


In [14]:
pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'])

C             bar       foo
A     B                    
one   A  0.320352  0.158178
      B  0.008458  1.245662
      C  1.115178 -0.424375
three A  0.314158       NaN
      B       NaN  1.628074
      C  0.291521       NaN
two   A       NaN -0.202699
      B -0.768427       NaN
      C       NaN  0.025907

In [15]:
pd.pivot_table(df, values='D', index=['B'], columns=['A', 'C'], aggfunc=np.sum)

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A  0.640705  0.316355  0.628317       NaN       NaN -0.405399
B  0.016916  2.491324       NaN  3.256147 -1.536854       NaN
C  2.230356 -0.848750  0.583041       NaN       NaN  0.051814

In [16]:
pd.pivot_table(df, values=['D','E'], index=['B'], columns=['A', 'C'], aggfunc=np.sum)

D                                                           E  \
A       one               three                 two                 one   
C       bar       foo       bar       foo       bar       foo       bar   
B                                                                         
A  0.640705  0.316355  0.628317       NaN       NaN -0.405399  0.902611   
B  0.016916  2.491324       NaN  3.256147 -1.536854       NaN -0.122489   
C  2.230356 -0.848750  0.583041       NaN       NaN  0.051814 -0.907992   

                                                     
A               three                 two            
C       foo       bar       foo       bar       foo  
B                                                    
A -1.220498 -0.062577       NaN       NaN  0.396079  
B -3.501791       NaN  0.574189  0.912127       NaN  
C -0.170606 -1.360093       NaN       NaN  1.640055

In [18]:
# Also, you can use Grouper for index and columns keywords.
pd.pivot_table(df, values='D', index=pd.Grouper(freq='M', key='F'), columns='C')

C,bar,foo
F,,
2013-01-31,NaN,0.158178
2013-02-28,NaN,1.245662
2013-03-31,NaN,0.025907
2013-04-30,0.314158,NaN
2013-05-31,0.008458,NaN
2013-06-30,1.115178,NaN
2013-07-31,NaN,-0.202699
2013-08-31,NaN,1.628074
2013-09-30,NaN,-0.424375


In [22]:
# render a nice output of the table omitting the missing values by calling to_string:
table = pd.pivot_table(df, index=['A', 'B'], columns=['C'])
print(table.to_string(na_rep=''))

                D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  0.320352  0.158178  0.451305 -0.610249
      B  0.008458  1.245662 -0.061244 -1.750896
      C  1.115178 -0.424375 -0.453996 -0.085303
three A  0.314158           -0.031288          
      B            1.628074            0.287095
      C  0.291521           -0.680046          
two   A           -0.202699            0.198040
      B -0.768427            0.456064          
      C            0.025907            0.820027


In [24]:
# If you pass margins=True to pivot_table, special All columns and rows will be added with 
# partial group aggregates across the categories on the rows and columns:
df.pivot_table(index=['A', 'B'], columns='C', margins=True, aggfunc=np.std)

D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  0.156674  1.941174  1.128273  1.045275  0.097979  0.861993
      B  1.320398  0.522962  1.087446  1.793702  1.221112  1.587808
      C  1.401589  0.695454  1.267324  1.278689  0.943581  0.941864
three A  1.693840       NaN  1.693840  0.989355       NaN  0.989355
      B       NaN  0.092394  0.092394       NaN  1.431882  1.431882
      C  0.423765       NaN  0.423765  0.590296       NaN  0.590296
two   A       NaN  1.212490  1.212490       NaN  0.093688  0.093688
      B  0.229833       NaN  0.229833  1.440580       NaN  1.440580
      C       NaN  0.015320  0.015320       NaN  1.560876  1.560876
All      0.978297  1.084311  0.993331  1.021743  1.165063  1.051334

can also provide a list of aggfunctions  
ex.   
``aggfunc={"Quantity":len,"Price":[np.sum,np.mean]}``